In [1]:
import requests
import json
import ollama
from IPython import display

In [2]:
text = """
Meta AI, đối thủ của ChatGPT, dự kiến được triển khai tại thị trường Việt Nam trong năm nay, "Chúng tôi tự hào là công ty công nghệ lớn duy nhất tại Thung lũng Silicon cung cấp công cụ AI miễn phí cho người dùng Việt", ông Nick Clegg, Chủ tịch phụ trách Đối ngoại toàn cầu Meta, nói tại buổi gặp gỡ truyền thông bên lề Ngày hội Đổi mới sáng tạo Việt Nam, sáng 1/10. Công cụ có tên Meta AI, hoạt động tương tự ChatGPT, nhưng sử dụng mô hình Llama do công ty này phát triển. Ông Clegg nhấn mạnh đây là mô hình ngôn ngữ lớn dạng mở, hỗ trợ tiếng Việt, giúp doanh nghiệp Việt không cần "tốn hàng tỷ USD để tự xây dựng mô hình ngôn ngữ lớn". Nick Clegg Chủ tịch phụ trách Đối ngoại Meta, tháng 10/2024. Ảnh: Lưu Quý Nick Clegg Chủ tịch phụ trách Đối ngoại Meta. Ảnh: Lưu Quý Đại diện Meta cũng cho biết AI dành cho doanh nghiệp trên Messenger đã được thử nghiệm tại Việt Nam từ tháng 6 và sẽ chính thức ra mắt cuối năm nay. Sản phẩm có thể được đưa lên các nền tảng như Messenger, WhatsApp. Cùng với AI, một trọng tâm khác trong chuyến thăm Việt Nam lần này của Chủ tịch đối ngoại Meta là việc chọn Việt Nam làm nơi sản xuất thiết bị thực tế ảo hỗn hợp Quest 3S, vừa được giới thiệu cuối tháng 9, có giá khởi điểm 300 USD. Việc mở rộng sản xuất tại Việt Nam dự kiến có thể tạo ra 1.000 việc làm và đóng góp hàng triệu USD cho nền kinh tế. "Việt Nam tiếp tục là một quốc gia quan trọng đối với Meta. Hàng triệu doanh nghiệp nhỏ và người tiêu dùng Việt đang sử dụng nền tảng của chúng tôi", ông Clegg nói. Ngoài ra, hãng công nghệ Mỹ cũng cho biết đã hợp tác với Đại học Khoa học Xã hội và Nhân văn - Đại học Quốc gia Hà Nội đào tạo tín chỉ về AI cho sinh viên, dự kiến triển khai vào tháng 3/2025. Chương trình nhằm nâng cao kỹ năng cho giáo viên và sinh viên, trải nghiệm AI một cách an toàn và có trách nhiệm. Logo Meta tại sự kiện ở Việt Nam, tháng 10/2024. Ảnh: Lưu Quý Logo Meta tại sự kiện ở Việt Nam, tháng 10/2024. Ảnh: Lưu Quý Meta là tập đoàn công nghệ đa quốc gia, phát triển các nền tảng mạng xã hội như Facebook, Instagram, Whatsapp, Threads, và các thiết bị phần cứng như kính VR/AR. Hãng tăng cường hiện diện tại Việt Nam từ năm 2015, đã phối hợp với các bộ, ngành triển khai nhiều hoạt động hợp tác trong lĩnh vực kinh tế số, đổi mới sáng tạo.
"""

llm_options = {
    "temperature": 0,
    "num_ctx": 2048,
    "num_predict": 4096,
    "top_k": 100, # Set from 1-100
    "top_p": 0.9 # Set from 0.1-0.9
}

llama_1b = "llama3.2:1b"
llama_3b = "llama3.2:3b"

In [3]:
# Call via REST API
def gen_content(
    prompts: [str],
    system_prompt: str=None,
    model: str="llama3.2",
    options: {}={}) -> str:
    url = "http://localhost:11434/api/chat"
    messages = []
    if system_prompt is not None:
        messages.append(
            {
                "role": "system",
                "content": system_prompt
            }
        )

    for prompt in prompts:
        messages.append(
            {
                "role": "user",
                "content": prompt
            }
        )
    data = {
        "model": model,
        "messages": messages,
        "stream": False,
        "options": options
    }
    headers = {
        "Content-Type": "application/json"
    }
    res = requests.post(
        url=url,
        headers=headers,
        json=data
    )
    return res.json()["message"]["content"]

In [4]:
response = gen_content(
    prompts=[text, "Tóm tắt văn bản trên:"],
    model=llama_1b,
    options=llm_options
)

display.Markdown(response)

Meta AI dự kiến được triển khai tại thị trường Việt Nam vào năm nay. Công cụ này có tên Meta AI và hoạt động tương tự ChatGPT nhưng sử dụng mô hình ngôn ngữ lớn Llama phát triển bởi Meta. Đại diện của công ty cho biết sản phẩm đã thử nghiệm tại Việt Nam từ tháng 6 và sẽ chính thức ra mắt cuối năm nay. Ngoài ra, Meta cũng hợp tác với Đại học Khoa học Xã hội và Nhân văn - Đại học Quốc gia Hà Nội để đào tạo tín chỉ về AI cho sinh viên.

In [5]:
def gen_content_api(
    prompts: [str],
    system_prompt: str=None,
    model: str="llama3.2",
    options: {}={}) -> str:
    messages = []
    if system_prompt is not None:
        messages.append(
            {
                "role": "system",
                "content": system_prompt
            }
        )

    for prompt in prompts:
        messages.append(
            {
                "role": "user",
                "content": prompt
            }
        )
    res = ollama.chat(
        model=model,
        messages=messages,
        options=options,
        stream=False
    )
    return res['message']['content']

In [6]:
response = gen_content_api(
    prompts=[text, "Tóm tắt văn bản trên:"],
    model=llama_3b,
    options=llm_options
)

display.Markdown(response)

Nick Clegg, Chủ tịch phụ trách Đối ngoại toàn cầu Meta, cho biết công cụ AI miễn phí của công ty sẽ được triển khai tại thị trường Việt Nam vào cuối năm nay. Công cụ hoạt động tương tự ChatGPT nhưng sử dụng mô hình Llama do Meta phát triển. Ngoài ra, Meta cũng cho biết đã hợp tác với Đại học Khoa học Xã hội và Nhân văn - Đại học Quốc gia Hà Nội đào tạo tín chỉ về AI cho sinh viên, dự kiến triển khai vào tháng 3/2025.

In [7]:
response = ollama.generate(
    model=llama_3b,
    prompt="Where is Tokyo",
    system="You are a 3 months old baby. You can only cry, make some random sounds, and smile.",
    options=llm_options
)

display.Markdown(response["response"])

WAAAAAAAHHH!!! *flails arms* Oooh oooh! *giggles* Ee ee ee! *smiles*